In [1]:
import pandas as pd
import numpy as np

In [2]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb(api_key='24a3062e32228f2015236a5f3597ca84')
# tmdb.api_key = '24a3062e32228f2015236a5f3597ca84'

## Extracting features of 2018, 2019, 2020, 2021, 2022 movies from Wikipedia

In [3]:
genres=eval(requests.get("https://api.themoviedb.org/3/genre/movie/list?api_key=24a3062e32228f2015236a5f3597ca84&language=en-US").content)['genres']

genres_map=pd.DataFrame(genres).set_index("id")

In [ ]:
#2018
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2018"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]
df_2018 = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)
df_2018['year']=2018

#2019
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2019"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]
df_2019 = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)
df_2019['year']=2019

#2020
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]
df_2020 = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)
df_2020 = df_2020[df_2020['Title'].apply(lambda x:x not in ("No films had been released on this date as resulting the closure of theaters caused by the COVID-19 pandemic","The SpongeBob Movie: Sponge on the Run (Canadian theatrical release)"))]
df_2020['year']=2020

#2021
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2021"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]
df_2021 = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)
df_2021=df_2021[~df_2021['Title'].isna()]
df_2021['year']=2021

#2022
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2022"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]
df_2022 = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)
df_2022=df_2022[10:]
df_2022['year']=2022

new_data_cols=['Title','Cast and crew','year']

new_data=df_2018.append(df_2019.append(df_2020.append(df_2021.append(df_2022,ignore_index=True),ignore_index=True),ignore_index=True),ignore_index=True)[new_data_cols]

def int_year(x):
    try:return int(x)
    except:return -99999

def get_details(x):
    try:
        year=x.release_date[:4]
    except:
        year=-99999
    return {"id":x.id,"title":x.title,"year":year,"genres":" ".join(map(lambda g:genres_map.loc[g][0],x.genre_ids))}

def search_movie(x,year=None):
    print(x,year)
    if year is None:
        return pd.DataFrame(map(get_details,tmdb.search(x))).sort_values("year",ascending=False)[["id","title","genres"]].loc[0].to_dict()
    else:
        search_df=pd.DataFrame(map(get_details,tmdb.search(x)))
        search_df['year_diff']=search_df['year'].apply(lambda y:abs(int_year(y)-year))
        return search_df.sort_values("year_diff",ascending=True)[["id","title","genres"]].iloc[0].to_dict()
    
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]
    
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])
    
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])



In [5]:
new_data=pd.concat([new_data,new_data.apply(lambda x:search_movie(x['Title'],x['year']),axis=1,result_type='expand')],axis='columns')

new_data['director_name'] = new_data['Cast and crew'].map(lambda x: get_director(str(x)))
new_data['actor_1_name'] = new_data['Cast and crew'].map(lambda x: get_actor1(x))
new_data['actor_2_name'] = new_data['Cast and crew'].map(lambda x: get_actor2(x))
new_data['actor_3_name'] = new_data['Cast and crew'].map(lambda x: get_actor3(x))
new_data['title']=new_data['Title'].str.lower()

/var/folders/8p/44mq6_9n6sl5sg2z924w6wkm0000gq/T/ipykernel_83899/1597204018.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2018 = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)
/var/folders/8p/44mq6_9n6sl5sg2z924w6wkm0000gq/T/ipykernel_83899/1597204018.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2019 = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)
/var/folders/8p/44mq6_9n6sl5sg2z924w6wkm0000gq/T/ipykernel_83899/1597204018.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2020 = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)
/var/folders/8p/44mq6_9n6sl5sg2z924

Insidious: The Last Key 2018
The Strange Ones 2018
Stratton 2018
Sweet Country 2018
The Commuter 2018
Proud Mary 2018
Acts of Violence 2018
Freak Show 2018
Humor Me 2018
Vazante 2018
Mary and the Witch's Flower 2018
12 Strong 2018
Den of Thieves 2018
Forever My Girl 2018
Maze Runner: The Death Cure 2018
The Insult 2018
Please Stand By 2018
Winchester 2018
A Fantastic Woman 2018
Armed 2018
The Cloverfield Paradox 2018
Bad Apples 2018
Peter Rabbit 2018
Fifty Shades Freed 2018
The 15:17 to Paris 2018
La Boda de Valentina 2018
Kri 2018
Permission 2018
Monster Family 2018
Golden Exits 2018
Black Panther 2018
Early Man 2018
Loveless 2018
The Party 2018
Nostalgia 2018
Black '47 2018
Samson 2018
Game Night 2018
Annihilation 2018
Every Day 2018
The Cured 2018
The Lodgers 2018
Red Sparrow 2018
Death Wish 2018
The Vanishing of Sidney Hall 2018
Foxtrot 2018
Pickings 2018
A Wrinkle in Time 2018
Gringo 2018
Thoroughbreds 2018
The Death of Stalin 2018
The Leisure Seeker 2018
The Strangers: Prey at Ni

The Farewell 2019
Darlin' 2019
The Lion King 2019
Once Upon a Time in Hollywood 2019
Skin 2019
The Red Sea Diving Resort 2019
Fast & Furious Presents: Hobbs & Shaw 2019
A Score to Settle 2019
The Operative 2019
Dora and the Lost City of Gold 2019
The Kitchen 2019
The Art of Racing in the Rain 2019
Scary Stories to Tell in the Dark 2019
The Peanut Butter Falcon 2019
After the Wedding 2019
Light of My Life 2019
The Angry Birds Movie 2 2019
Good Boys 2019
Blinded by the Light 2019
Where'd You Go, Bernadette 2019
47 Meters Down: Uncaged 2019
Gwen 2019
Ready or Not 2019
Angel Has Fallen 2019
Brittany Runs a Marathon 2019
Overcomer 2019
Jacob's Ladder 2019
Burn 2019
Official Secrets 2019
Don't Let Go 2019
Itsy Bitsy 2019
The Fanatic 2019
It Chapter Two 2019
Satanic Panic 2019
Strange but True 2019
The Obituary of Tunde Johnson 2019
The Goldfinch 2019
Hustlers 2019
The Sound of Silence 2019
Monos 2019
Tall Girl 2019
Ad Astra 2019
Downton Abbey 2019
Rambo: Last Blood 2019
Bloodline 2019
Runnin

The Marksman 2021
The Ultimate Playlist of Noise 2021
American Skin 2021
Don't Tell a Soul 2021
Our Friend 2021
No Man's Land 2021
Born a Champion 2021
Wrong Turn 2021
Penguin Bloom 2021
The Little Things 2021
Finding ʻOhana 2021
Nomadland 2021
Malcolm & Marie 2021
Palmer 2021
Falling 2021
Bliss 2021
Little Fish 2021
Son of the South 2021
PVT Chat 2021
Music 2021
Judas and the Black Messiah 2021
The Mauritanian 2021
Breaking News in Yuba County 2021
Barb and Star Go to Vista Del Mar 2021
Minari 2021
To All the Boys: Always and Forever 2021
The World to Come 2021
French Exit 2021
Land 2021
Willy's Wonderland 2021
The Map of Tiny Perfect Things 2021
Fear of Rain 2021
I Care a Lot 2021
Flora & Ulysses 2021
Silk Road 2021
Tom & Jerry 2021
The United States vs. Billie Holiday 2021
Cherry 2021
Crisis 2021
Billie Eilish: The World's a Little Blurry 2021
The Vigil 2021
Moxie 2021
Coming 2 America 2021
The SpongeBob Movie: Sponge on the Run 2021
Raya and the Last Dragon 2021
Chaos Walking 2021


Fresh 2022
A Day to Die 2022
The Adam Project 2022
Turning Red 2022
Tyson's Run 2022
Rescued by Ruby 2022
The Outfit 2022
Deep Water 2022
Cheaper by the Dozen 2022
Windfall 2022
X 2022
Umma 2022
Alice 2022
Master 2022
Measure of Revenge 2022
The Lost City 2022
Everything Everywhere All at Once 2022
Infinite Storm 2022
7 Days 2022
Moonshot 2022
Morbius 2022
The Bubble 2022
The Contractor 2022
Better Nate Than Ever 2022
Apollo 10 1⁄2: A Space Age Childhood 2022
Sonic the Hedgehog 2 2022
Ambulance 2022
All the Old Knives 2022
Metal Lords 2022
Father Stu 2022
Fantastic Beasts: The Secrets of Dumbledore 2022
Dual 2022
Choose or Die 2022
Titanic 666 2022
The Bad Guys 2022
The Northman 2022
The Unbearable Weight of Massive Talent 2022
Polar Bear 2022
The Survivor 2022
Memory 2022
Crush 2022
Fortress: Sniper's Eye 2022
Green Ghost and the Masters of the Stone 2022
Doctor Strange in the Multiverse of Madness 2022
Along for the Ride 2022
Marmaduke 2022
Escape the Field 2022
Operation Mincemeat 2

In [6]:
new_data['actor_2_name'] = new_data['actor_2_name'].replace(np.nan, 'unknown')
new_data['actor_3_name'] = new_data['actor_3_name'].replace(np.nan, 'unknown')

In [7]:
new_data['movie_title'] = new_data['Title'].str.lower()

In [8]:
new_data['comb'] = new_data['actor_1_name'] + ' ' + new_data['actor_2_name'] + ' '+ new_data['actor_3_name'] + ' '+ new_data['director_name'] +' ' + new_data['genres']

In [9]:
new_data

,Title,Cast and crew,year,id,title,genres,director_name,actor_1_name,actor_2_name,actor_3_name,movie_title,comb
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,2018,406563,insidious: the last key,Horror Mystery Thriller,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell,insidious: the last key,Lin Shaye Angus Sampson Leigh Whannell Adam Ro...
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,2018,426258,the strange ones,Thriller Drama,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus,the strange ones,Alex Pettyfer James Freedson-Jackson Emily Alt...
2,Stratton,"Simon West (director); Duncan Falconer, Warren...",2018,348389,stratton,Action Thriller,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan,stratton,Dominic Cooper Austin Stowell Gemma Chan Simon...
3,Sweet Country,"Warwick Thornton (director); David Tranter, St...",2018,468210,sweet country,Drama History Western,Warwick Thornton,Bryan Brown,Sam Neill,unknown,sweet country,Bryan Brown Sam Neill unknown Warwick Thornton...
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,2018,399035,the commuter,Action Thriller Mystery,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson,the commuter,Liam Neeson Vera Farmiga Patrick Wilson Jaume ...
...,...,...,...,...,...,...,...,...,...,...,...,...
1366,Luckiest Girl Alive,Mike Barker (director); Jessica Knoll (screenp...,2022,799546,luckiest girl alive,Drama Thriller Mystery,Mike Barker,Mila Kunis,Finn Wittrock,Scoot McNairy,luckiest girl alive,Mila Kunis Finn Wittrock Scoot McNairy Mike Ba...
1367,God's Creatures,"Saela Davis, Anna Rose Holmer (directors); Sha...",2022,828558,god's creatures,Drama,"Saela Davis, Anna Rose Holmer",Emily Watson,Paul Mescal,Aisling Franciosi,god's creatures,Emily Watson Paul Mescal Aisling Franciosi Sae...
1368,Dead for a Dollar,Walter Hill (director/screenplay); Christoph W...,2022,843380,dead for a dollar,Western Thriller,Walter Hill,Christoph Waltz,Willem Dafoe,Rachel Brosnahan,dead for a dollar,Christoph Waltz Willem Dafoe Rachel Brosnahan ...
1369,My Best Friend's Exorcism,Damon Thomas (director); Jenna Lamia (screenpl...,2022,816952,my best friend's exorcism,Horror Comedy,Damon Thomas,Elsie Fisher,Amiah Miller,Cathy Ang,my best friend's exorcism,Elsie Fisher Amiah Miller Cathy Ang Damon Thom...


In [10]:
old_df = pd.read_csv('../datasets/new_data.csv')

In [11]:
old_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,year,id,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,2009.0,19995,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,2007.0,285,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,2015.0,206647,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,2012.0,49026,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens,NaN,140607,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...,...,...
5341,Jim Strouse,Jessica Williams,Chris O'Dowd,Keith Stanfield,Romance Comedy,the incredible jessica james,NaN,432789,Jessica Williams Chris O'Dowd Keith Stanfield ...
5342,Farhad Mann,Adelaide Kane,Benjamin Hollingsworth,Jean Louisa Kelly,Romance,can't buy my love,NaN,445126,Adelaide Kane Benjamin Hollingsworth Jean Loui...
5343,Hannaleena Hauru,Inka Haapamäki,Rosa Honkonen,Tiitus Rantala,Romance Comedy,thick lashes of lauri mäntyvaara,NaN,468707,Inka Haapamäki Rosa Honkonen Tiitus Rantala Ha...
5344,Jonathan A. Rosenbaum,Lou Diamond Phillips,Wallace Shawn,Gina Holden,Crime Comedy Action Family,cop and a half: new recruit,NaN,461297,Lou Diamond Phillips Wallace Shawn Gina Holden...


In [12]:
final_df = old_df.append(new_data,ignore_index=True)[['id','movie_title','year','actor_1_name','actor_2_name','actor_3_name','director_name','genres','comb']]

/var/folders/8p/44mq6_9n6sl5sg2z924w6wkm0000gq/T/ipykernel_83899/1247262102.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = old_df.append(new_data,ignore_index=True)[['id','movie_title','year','actor_1_name','actor_2_name','actor_3_name','director_name','genres','comb']]


In [13]:
final_df

,id,movie_title,year,actor_1_name,actor_2_name,actor_3_name,director_name,genres,comb
0,19995,avatar,2009.0,CCH Pounder,Joel David Moore,Wes Studi,James Cameron,Action Adventure Fantasy Sci-Fi,CCH Pounder Joel David Moore Wes Studi James C...
1,285,pirates of the caribbean: at world's end,2007.0,Johnny Depp,Orlando Bloom,Jack Davenport,Gore Verbinski,Action Adventure Fantasy,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,206647,spectre,2015.0,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Sam Mendes,Action Adventure Thriller,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,49026,the dark knight rises,2012.0,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Christopher Nolan,Action Thriller,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,140607,star wars: episode vii - the force awakens,NaN,Doug Walker,Rob Walker,unknown,Doug Walker,Documentary,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...,...,...
6712,799546,luckiest girl alive,2022.0,Mila Kunis,Finn Wittrock,Scoot McNairy,Mike Barker,Drama Thriller Mystery,Mila Kunis Finn Wittrock Scoot McNairy Mike Ba...
6713,828558,god's creatures,2022.0,Emily Watson,Paul Mescal,Aisling Franciosi,"Saela Davis, Anna Rose Holmer",Drama,Emily Watson Paul Mescal Aisling Franciosi Sae...
6714,843380,dead for a dollar,2022.0,Christoph Waltz,Willem Dafoe,Rachel Brosnahan,Walter Hill,Western Thriller,Christoph Waltz Willem Dafoe Rachel Brosnahan ...
6715,816952,my best friend's exorcism,2022.0,Elsie Fisher,Amiah Miller,Cathy Ang,Damon Thomas,Horror Comedy,Elsie Fisher Amiah Miller Cathy Ang Damon Thom...


In [14]:
final_df.isna().sum()

id                 0
movie_title        0
year             548
actor_1_name       0
actor_2_name       0
actor_3_name       0
director_name      0
genres             0
comb               0
dtype: int64

In [15]:
final_df.drop_duplicates(subset ="id", keep = 'last', inplace = True)

In [16]:
final_df

,id,movie_title,year,actor_1_name,actor_2_name,actor_3_name,director_name,genres,comb
0,19995,avatar,2009.0,CCH Pounder,Joel David Moore,Wes Studi,James Cameron,Action Adventure Fantasy Sci-Fi,CCH Pounder Joel David Moore Wes Studi James C...
1,285,pirates of the caribbean: at world's end,2007.0,Johnny Depp,Orlando Bloom,Jack Davenport,Gore Verbinski,Action Adventure Fantasy,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,206647,spectre,2015.0,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Sam Mendes,Action Adventure Thriller,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,49026,the dark knight rises,2012.0,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Christopher Nolan,Action Thriller,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,140607,star wars: episode vii - the force awakens,NaN,Doug Walker,Rob Walker,unknown,Doug Walker,Documentary,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...,...,...
6712,799546,luckiest girl alive,2022.0,Mila Kunis,Finn Wittrock,Scoot McNairy,Mike Barker,Drama Thriller Mystery,Mila Kunis Finn Wittrock Scoot McNairy Mike Ba...
6713,828558,god's creatures,2022.0,Emily Watson,Paul Mescal,Aisling Franciosi,"Saela Davis, Anna Rose Holmer",Drama,Emily Watson Paul Mescal Aisling Franciosi Sae...
6714,843380,dead for a dollar,2022.0,Christoph Waltz,Willem Dafoe,Rachel Brosnahan,Walter Hill,Western Thriller,Christoph Waltz Willem Dafoe Rachel Brosnahan ...
6715,816952,my best friend's exorcism,2022.0,Elsie Fisher,Amiah Miller,Cathy Ang,Damon Thomas,Horror Comedy,Elsie Fisher Amiah Miller Cathy Ang Damon Thom...


In [17]:
final_df.to_csv('../datasets/final_data.csv',index=False)